In [26]:
import numpy as np

def compute_K_vectors(as_param, aq_param, cs_param, cq_param, rho_s, rho_q, sigma_squared, eta):
    """
    Compute the K vectors as defined in Equation 22
    
    Parameters:
    -----------
    as_param, aq_param: Parameters for the short-run and long-run volatility components
    cs_param, cq_param: Leverage parameters for the short-run and long-run components
    rho_s, rho_q: Persistence parameters for short-run and long-run components
    sigma_squared: Unconditional variance
    eta: Parameter for IG innovations
    
    Returns:
    --------
    K1, K2, K3, K4, K5: The K vectors as defined in Equation 22
    """

    # Define K2 and K3
    K2 = np.array([as_param, aq_param])
    K3 = np.array([cs_param, cq_param])
    
    # Compute K1
    K1 = np.array([0, sigma_squared * (1 - rho_q)]) - eta**4 * K2
    
    # Compute K4
    K4 = np.array([rho_s, 0]) - eta**2 * K2 - (1/eta**2) * K3
    
    # Compute K5
    K5 = np.array([0, rho_q]) - eta**2 * K2 - (1/eta**2) * K3
    
    return K1, K2, K3, K4, K5

def compute_kappa_values(theta_l, theta_y, K2, K3, eta):
    """
    Compute κ₁ and κ₂ as defined in Equations 25 and 26
    
    Parameters:
    -----------
    theta_l: Vector of factor risk preference parameters [theta_s, theta_q]
    theta_y: Equity risk preference parameter
    K2, K3: K vectors from Equation 22
    eta: Parameter for IG innovations
    lambda_param: Equity risk premium parameter
    
    Returns:
    --------
    kappa1, kappa2: Values of κ₁ and κ₂
    """
    # Calculate kappa1 (Equation 25)
    kappa1 = 1 - 2 * eta**4 * np.dot(theta_l, K2)
    
    # Calculate kappa2 (Equation 26)
    kappa2 = 1 - 2 * theta_y * eta - 2 * np.dot(theta_l, K3)
    
    return kappa1, kappa2

In [29]:
def compute_theta_l(theta_s, theta_q):
    """
    Compute θl based on Equation 21 from the JF paper
    
    Parameters:
    -----------
    theta_s, theta_q : float
        Factor risk preference parameters
    
    Returns:
    --------
    theta_l : array-like
        Vector of factor risk preference parameters [theta_s, theta_q]
    """
    theta_l = np.array([theta_s, theta_q])
    return theta_l

def compute_theta_y(theta_l, lambda_param, eta, K2, K3):
    """
    Compute θy based on Equation 23 from the JF paper
    
    Parameters:
    -----------
    theta_l : array-like
        Vector of factor risk preference parameters [theta_s, theta_q]
    lambda_param : float
        Equity risk premium parameter
    eta : float
        Parameter for IG innovations
    K2, K3 : array-like
        K vectors from Equation 22
    
    Returns:
    --------
    theta_y : float
        Equity risk preference parameter
    """
    # Term 1: (1 - 2θl^T K3)/(2η)
    term1 = (1 - 2 * np.dot(theta_l, K3)) / (2 * eta)
    
    num = lambda_param**2 * eta**3 + 2 * (1 - 2 * eta**4 * np.dot(theta_l, K2))
    denom = 8 * lambda_param**2 * eta**3 * (1 - 2 * eta**4 * np.dot(theta_l, K2))
    term2 = num**2 / denom
    
    # Combine the terms to get θy
    theta_y = term1 - term2
    
    return theta_y

In [19]:
def compute_model_params_and_vectors(eta, lam, K1, K2, K3, K4, K5, kappa1, kappa2):
    """
    Compute the model parameters and vectors for IG_GARCH(C)
    
    Parameters:
    -----------
    eta: Parameter for IG innovations
    lam: Equity risk premium parameter
    K1, K2, K3, K4, K5: K vectors from Equation 22
    kappa1, kappa2: Values of κ₁ and κ₂ from Equations 25 and 26
    
    Returns:
    --------
    eta*, lambda*, K1*, K2*, K3*, K4*, K5*: Model parameters and vectors for IG_GARCH(C)
    """
    
    # Compute eta*
    eta_star = eta / kappa2

    # Compute lambda*
    lam_star = lam * np.sqrt((kappa2**3)/kappa1)

    # Compute K1*
    K1_star = K1 * np.sqrt(kappa1/(kappa2**3))

    # Compute K2*
    K2_star = K2 * np.sqrt((kappa2**5)/kappa1)

    # Compute K3*
    K3_star = K3 * np.sqrt(kappa1/(kappa2**5))

    # Compute K4*
    K4_star = K4

    # Compute K5*
    K5_star = K5
    
    return eta_star, lam_star, K1_star, K2_star, K3_star, K4_star, K5_star

In [20]:
import functools # for memoizing the recursive functions for B_s* and B_q*

def compute_B_star(u, v, t, T, **params):
    """
    Compute the B* function for IG_GARCH(C)
    
    Parameters:
    -----------
    u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
    t, T: Time periods
    
    Returns:
    --------
    B_star: Vector generated from B_s* and B_q* components
    """
    
    if t == T:
        return v
    else:
        return np.array(compute_B_s_star(u, v, t, T, **params) + compute_B_q_star(u, v, t, T, **params))

@functools.lru_cache(maxsize=None)
def compute_B_s_star(u, v, t, T, **params):
    """
    Compute the B_s* function for IG_GARCH(C)
    
    Parameters:
    -----------
    u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
    t, T: Time periods
    params: 
    ---------
    Dictionary of model parameters
        eta_star: Parameter for IG innovations
        lam_star: Equity risk premium parameter
        K1_star, K2_star, K3_star, K4_star, K5_star: risk free K vectors for IG_GARCH(C)
    
    Returns:
    --------
    B_s_star: Value of the B_s* function
    """
    
    if t == T:
        return 0
    else:
        eta_star = params['eta_star']
        lam_star = params['lam_star']
        K2_star = params['K2_star']
        K3_star = params['K3_star']
        K4_star = params['K4_star']

        Bstar_tplus1 = compute_B_star(u, v, t+1, T, **params)

        term1 = 1 - 2 * eta_star**4 * np.dot(Bstar_tplus1, K2_star)
        term2 = 1 - 2 * u * eta_star - 2 * np.dot(Bstar_tplus1, K3_star)
        root = np.sqrt(term1 * term2)

        return u*lam_star + np.dot(Bstar_tplus1, K4_star) + (1 - root)/eta_star**2
    

@functools.lru_cache(maxsize=None)
def compute_B_q_star(u, v, t, T, **params):
    """
    Compute the B_q* function for IG_GARCH(C)
    
    Parameters:
    -----------
    u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
    t, T: Time periods
    params: 
    ---------
    Dictionary of model parameters
        eta_star: Parameter for IG innovations
        lam_star: Equity risk premium parameter
        K1_star, K2_star, K3_star, K4_star, K5_star: risk free K vectors for IG_GARCH(C)
    
    Returns:
    --------
    B_q_star: Value of the B_q* function
    """
    
    if t == T:
        return 0
    else:
        eta_star = params['eta_star']
        lam_star = params['lam_star']
        K2_star = params['K2_star']
        K3_star = params['K3_star']
        K5_star = params['K5_star']

        Bstar_tplus1 = compute_B_star(u, v, t+1, T, **params)

        term1 = 1 - 2 * eta_star**4 * np.dot(Bstar_tplus1, K2_star)
        term2 = 1 - 2 * u * eta_star - 2 * np.dot(Bstar_tplus1, K3_star)
        root = np.sqrt(term1 * term2)

        return u*lam_star + np.dot(Bstar_tplus1, K5_star) + (1 - root)/eta_star**2
    

def compute_B_0_star(u, v, t, T, **params):
    return 0



In [21]:
@functools.lru_cache(maxsize=None)
def compute_A_star(u, v, t, T, **params):
    """
    Compute the A* function for IG_GARCH(C)
    
    Parameters:
    -----------
    u, v: Equity and Model Factor Risk Preference Parameters for IG-GARCH(C)
    t, T: Time periods
    params: 
    ---------
    Dictionary of model parameters
        eta_star: Parameter for IG innovations
        lam_star: Equity risk premium parameter
        K1_star, K2_star, K3_star, K4_star, K5_star: risk free K vectors for IG_GARCH(C)
        r: Risk-free rate
    
    Returns:
    --------
    A_star: Value of A*
    """

    eta_star = params['eta_star']
    K1_star = params['K1_star']
    K2_star = params['K2_star']
    r = params["r"]

    if t == T:
        return 0
    else:
        Astar_tplus1 = compute_A_star(u, v, t+1, T, **params)
        Bstar_tplus1 = compute_B_star(u, v, t+1, T, **params)
        log_term = 1 - 2*eta_star**4 * np.dot(Bstar_tplus1, K2_star)
        return Astar_tplus1 + u*r + np.dot(Bstar_tplus1, K1_star) - 0.5 * np.log(log_term)
        

In [22]:
def psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star):
    """
    Compute psi_0_t value as defined in Proposition 2
    """
    # Computing A_star(2, 0; t, t+1)
    A_star_val = A_star(2, 0, t, t+1)
    
    # Computing B0_star(2, 0; t, t+1)
    B0_star_val = B0_star(2, 0, t, t+1)
    
    # Computing B_star(2, 0; t, t+1)
    B_star_val = B_star(2, 0, t, t+1)
    
    # Handle both array and scalar cases for B_star_val
    if hasattr(B_star_val, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val * l_star_t[0]
    
    # Computing psi_0_t
    psi_0_t = np.exp(-2*r + A_star_val + B0_star_val * y_t + dot_product) - 1
    
    return psi_0_t

def psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T):
    """
    Compute psi_1_t(z) value as defined in Proposition 2
    """
    # Computing A_star(z, 0; t, T)
    A_star_val = A_star(z, 0, t, T)
    
    # Computing B0_star(z, 0; t, T)
    B0_star_val = B0_star(z, 0, t, T)
    
    # Computing B_star(z, 0; t, T)
    B_star_val = B_star(z, 0, t, T)
    
    # Computing psi_1_t(z)
    # Handle both array and scalar cases for B_star_val
    if hasattr(B_star_val, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val * l_star_t[0]
        
    psi_1_val = np.exp(A_star_val + B0_star_val * y_t + dot_product)
    
    return psi_1_val

def psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T):
    """
    Compute psi_2_t(z) value as defined in Proposition 2
    """
    # Computing A_star(z, 0; t+1, T)
    A_star_val_1 = A_star(z, 0, t+1, T)
    
    # Computing B0_star(z, 0; t+1, T)
    B0_star_val = B0_star(z, 0, t+1, T)
    
    # Computing B_star(z, 0; t+1, T)
    B_star_val = B_star(z, 0, t+1, T)
    
    # Computing A_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    A_star_val_2 = A_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B0_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B0_star_val_2 = B0_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Computing B_star(1 + z + B0_star(z, 0; t+1, T), B_star(z, 0; t+1, T); t, t+1)
    B_star_val_2 = B_star(1 + z + B0_star_val, B_star_val, t, t+1)
    
    # Handle both array and scalar cases for B_star_val_2
    if hasattr(B_star_val_2, 'T'):  # If it's an array with transpose attribute
        dot_product = np.dot(B_star_val_2.T, l_star_t)
    else:  # If it's a scalar
        dot_product = B_star_val_2 * l_star_t[0]
    
    # Computing psi_2_t(z)
    psi_2_val = np.exp(-r + A_star_val_1 + A_star_val_2) * \
                np.exp(B0_star_val_2 * y_t) * \
                np.exp(dot_product)
    
    return psi_2_val

In [23]:
def integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T):
    """
    Compute the integrand for equation (8)
    
    Parameters:
    z: complex variable for the integration
    t: time point
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    
    Returns:
    Integrand value
    """
    # Computing e^((z-1)*Y_t)
    exp_term = np.exp((z-1) * y_t)
    
    # Computing psi_2_t(z) - psi_1_t(z)
    psi_diff = psi_2(t, z, y_t, l_star_t, r, A_star, B0_star, B_star, T) - \
               psi_1(t, z, y_t, l_star_t, A_star, B0_star, B_star, T)
    
    # Computing f_check(z)
    f_check_val = f_check(z, **option_params)
    
    # Computing the integrand
    result = exp_term * psi_diff * f_check_val
    
    return result

In [25]:
def f_check_call(z, K):
    """
    Inverse Laplace transform of a European call option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1))

def f_check_put(z, K):
    """
    Inverse Laplace transform of a European put option payoff
    
    Parameters:
    z: complex variable for the integration
    K: strike price
    
    Returns:
    Inverse Laplace transform value
    """
    return K**(1-z) / (z * (z - 1)) * (-1)  # The put option payoff has opposite sign

In [24]:
from scipy import integrate

def risk_minimizing_hedge(t, T, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, R):
    """
    Compute the risk-minimizing hedging position using equation (8)
    
    Parameters:
    t: current time point
    T: option maturity
    y_t: log-return at time t
    l_star_t: scaled factor vector at time t
    r: risk-free rate
    A_star, B0_star, B_star: Functions to compute the coefficients
    f_check: Inverse Laplace transform of the option payoff
    option_params: Parameters for the option payoff (e.g., strike price)
    R: Real part of the contour in the complex plane
    
    Returns:
    Risk-minimizing hedging position
    """
    # Computing psi_0_t
    psi_0_t = psi_0(t, y_t, l_star_t, r, A_star, B0_star, B_star)
    
    # Defining the complex contour for numerical integration
    # We'll use a contour that goes vertically through R + i*y for y in [-N, N]
    N = 100  # Limit for numerical integration
    
    # Defining the integrand function for the given parameters
    def integrand_for_quad(y):
        z = complex(R, y)
        return integrand(z, t, y_t, l_star_t, r, A_star, B0_star, B_star, f_check, option_params, T).real
    
    # Computing the integral using numerical integration
    integral_result, _ = integrate.quad(integrand_for_quad, -N, N)
    
    # Computing the final result with the complex i in the denominator
    # Since we're taking the real part of the integrand, we need to account for the i in the denominator
    # When dividing by i, it's equivalent to multiplying by -i
    # This rotates the result by -90 degrees in the complex plane
    xi_t_plus_1 = np.exp(-r * (T - t)) / (2 * np.pi * psi_0_t) * (-1) * integral_result
    
    return xi_t_plus_1